In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
env = gym.make('CartPole-v1')

CartPoleEnv - Version 0.2.0, Noise case: 1


/home/anany/anaconda3/envs/ML/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Environment '<class 'gym.envs.classic_control.cartpole.CartPoleEnv'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
#parameters
obs_space = len(env.observation_space.high)
buckets = (20, 500, 20, 500)
LEARNING_RATE = 0.01
EPISODES = 3000
DISCOUNT = 0.997
MAX_FRAMES = 500
act_space = env.action_space.n
bucket_size = env.observation_space.high/buckets - env.observation_space.low/buckets
q_table = np.random.randint(low=-500, high=0, size=(buckets + (act_space,)))

In [4]:
def bucket_no(state):
    bucket = state/bucket_size - env.observation_space.low/bucket_size
    return tuple(bucket.astype(np.int64))

In [5]:
def plot(scores=[]):
    plt.clf()
    avg_scores = [sum(scores[:index+1])/(index+1) for index, 
                  score in enumerate(scores)]
    avg_score = sum(scores)/len(scores)
    x_val = [0,len(scores)-1]
    y_val = [avg_score, avg_score]
    plt.plot(scores,'g-', label='current score')
    plt.plot(x_val, y_val,'r-', label='average score')
    plt.plot(avg_scores,'b-', label='average scores')
    plt.xlabel('Episodes--->')
    plt.ylabel('Score--->')
    plt.legend()
    plt.pause(0.01)

In [6]:
def next_action(state):
    return np.argmax(q_table[bucket_no(state)])

In [7]:
def play():
    score_list = np.array([])
    for episode in range(EPISODES):
        state = env.reset()
        score = 0
        for frame in range(MAX_FRAMES):
            step = next_action(state)
            next_state, reward, done, info = env.step(step)
            if done:
                q_table[bucket_no(state) + (step,)] = -500 + score
                break
            qt_current = q_table[bucket_no(state) + (step,)]
            max_qt_poss = np.max(q_table[bucket_no(next_state)])
            new_q = (1 - LEARNING_RATE) * qt_current + LEARNING_RATE * (score + DISCOUNT * max_qt_poss)
            q_table[bucket_no(state) + (step,)] = new_q
            state = next_state
            score += reward
        score_list = np.concatenate((score_list, score), axis=None)
        if not episode % 500:
            plot(score_list)
    env.close()

In [8]:
play()